In [10]:
# imports 
import numpy as np
import requests
import pandas as pd
import hashlib
from pathlib import Path

In [2]:
# data collection & acquisition 

# Chicago food inspection dataset acquisition
# access data via API and verify integrity with SHA-256

import requests
import pandas as pd
import hashlib
from pathlib import Path

OUTPUT_DIR = Path("data/raw")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# SODA2 version, does not require authentication
API_ENDPOINT = "https://data.cityofchicago.org/resource/4ijn-s7e5.json"

all_records = []
limit = 50000  
offset = 0

while True:
    params = {
        "$limit": limit,
        "$offset": offset,
        "$order": "inspection_date DESC"
    }
    print(f"Fetching records {offset} to {offset + limit}...")
    resp = requests.get(API_ENDPOINT, params=params)
    resp.raise_for_status()
    payload = resp.json()
    
    if len(payload) == 0:
        break
    
    all_records.extend(payload)
    print(f"  Retrieved {len(payload)} records (Total so far: {len(all_records)})")

    if len(payload) < limit:
        break
    
    offset += limit
print(f"\nSuccessfully fetched {len(all_records)} total records")

chicago_food_inspection = pd.json_normalize(all_records)
chicago_food_inspection.head()

csv_filename = OUTPUT_DIR / "food_inspections.csv"
chicago_food_inspection.to_csv(csv_filename, index=False)

# calculate SHA-256 checksum
with open(csv_filename, "rb") as f:
    sha = hashlib.sha256(f.read()).hexdigest()

sha_filename = OUTPUT_DIR / "food_inspections.sha256"
with open(sha_filename, "w", encoding="utf-8") as f:
    f.write(sha)  

# Verify the checksum was saved correctly
with open(sha_filename, "r", encoding="utf-8") as f:
    saved_checksum = f.read()

Fetching records 0 to 50000...
  Retrieved 50000 records (Total so far: 50000)
Fetching records 50000 to 100000...
  Retrieved 50000 records (Total so far: 100000)
Fetching records 100000 to 150000...
  Retrieved 50000 records (Total so far: 150000)
Fetching records 150000 to 200000...
  Retrieved 50000 records (Total so far: 200000)
Fetching records 200000 to 250000...
  Retrieved 50000 records (Total so far: 250000)
Fetching records 250000 to 300000...
  Retrieved 50000 records (Total so far: 300000)
Fetching records 300000 to 350000...
  Retrieved 72 records (Total so far: 300072)

Successfully fetched 300072 total records


In [3]:
chicago_food_inspection.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location.latitude,location.longitude,location.human_address,violations
0,2626789,FLOWERS NUTRITION LLC,FLOWERS NUTRITION (HERBALIFE),2873170,Restaurant,Risk 2 (Medium),3332 W MONTROSE AVE,CHICAGO,IL,60618,2025-11-07T00:00:00.000,Canvass Re-Inspection,Pass,41.96125126209583,-87.7120521820261,41.96125126209583,-87.7120521820261,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN
1,2626760,7-ELEVEN #33753H,7-ELEVEN,3056124,Grocery Store,Risk 2 (Medium),5320 N CUMBERLAND AVE,CHICAGO,IL,60656,2025-11-07T00:00:00.000,License Re-Inspection,Pass,41.97626545131273,-87.83676599105188,41.97626545131273,-87.83676599105188,"{""address"": """", ""city"": """", ""state"": """", ""zip""...","53. TOILET FACILITIES: PROPERLY CONSTRUCTED, S..."
2,2626761,7-ELEVEN #33753H,7-ELEVEN,3056125,Grocery Store,Risk 3 (Low),5320 N CUMBERLAND AVE,CHICAGO,IL,60656,2025-11-07T00:00:00.000,License Re-Inspection,Pass,41.97626545131273,-87.83676599105188,41.97626545131273,-87.83676599105188,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN
3,2626682,G & N SNACK SHOP,G & N THE KING OF BREAKFAST,3833,Restaurant,Risk 1 (High),5100 W NORTH AVE,CHICAGO,IL,60639,2025-11-06T00:00:00.000,Canvass,Pass w/ Conditions,41.90964983634534,-87.75269216065263,41.90964983634534,-87.75269216065263,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",8. HANDS CLEAN & PROPERLY WASHED - Comments: O...
4,2626683,RAYITO DE SOL SPANISH IMMERSION EARLY LEARNING...,RAYITO DE SOL,3051768,Daycare (2 - 6 Years),Risk 1 (High),1457 N HALSTED,CHICAGO,IL,60642,2025-11-06T00:00:00.000,License,Pass,41.908002954607774,-87.64805061743297,41.908002954607774,-87.64805061743297,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN


In [7]:
# Zillow Home Value Index dataset acquisition
# directly download the csv file 
# data available through 2025-09-30
csv_url = "https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1762826225"
response = requests.get(csv_url)

csv_filename = OUTPUT_DIR / "zhvi.csv"
with open(csv_filename, "wb") as f:
    f.write(response.content)
zhvi = pd.read_csv(csv_filename)

with open(csv_filename, "rb") as f:
    sha = hashlib.sha256(f.read()).hexdigest()

sha_filename = OUTPUT_DIR / "zhvi.sha256"
with open(sha_filename, "w", encoding="utf-8") as f:
    f.write(sha)

In [8]:
zhvi.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,211260.036103,...,503615.315072,504041.821563,503718.362554,502808.007227,501009.557989,498808.457699,496204.055384,494124.281431,493001.330687,493204.183103
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,117013.178176,...,546231.311054,546796.399030,548762.235908,551106.382435,554471.978686,557358.140113,559986.738688,561842.069988,563135.564098,565788.741081
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103949.915049,...,280364.214678,279752.347810,279127.333004,278293.728257,277333.261939,276527.656740,275525.866223,274583.243231,273646.325157,272866.097556
3,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,174013.827521,...,534715.303771,532223.576798,532092.170856,531746.686451,531633.596410,532026.401149,534097.682017,536855.532134,538527.597608,539769.173343
4,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102741.047530,...,273881.498841,273638.371800,273235.446503,272375.802036,271334.625636,270440.150302,269437.236297,268523.691827,267647.045438,266768.667034


# Data Acquisition Documentation

## Overview
This document provides detailed instructions for acquiring the two datasets used in this project. Each dataset uses a different acquisition method to demonstrate diverse data collection approaches:

1. **Chicago Food Inspections**: API access (Socrata SODA API)
2. **Zillow Home Value Index (ZHVI)**: Direct CSV download

Both datasets include SHA-256 checksums for integrity verification and reproducibility.


Or let Python create it automatically (included in notebooks).

---

## Dataset 1: Chicago Food Inspections

### Source Information
- **Provider**: City of Chicago Data Portal
- **Dataset Name**: Food Inspections
- **Dataset ID**: 4ijn-s7e5
- **URL**: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/about_data
- **API Endpoint**: https://data.cityofchicago.org/resource/4ijn-s7e5.json
- **License**: Public Domain (City of Chicago Open Data License)
- **Update Frequency**: Daily
- **Last Accessed**: 11/10/2025

### Dataset Description
This dataset contains records of inspections of food establishments in Chicago from January 1, 2010 to present. Inspections are performed by the Chicago Department of Public Health's Food Protection Program using a standardized procedure. Each inspection record includes:

- **Facility Information**: Business name (DBA), address, facility type
- **Risk Classification**: Risk 1 (High), Risk 2 (Medium), Risk 3 (Low)
- **Inspection Details**: Date, type, results (Pass/Fail/Conditional)
- **Geographic Data**: ZIP code, latitude, longitude
- **Violations**: Detailed description of any violations found

### Acquisition Method: API Access

**Why API?**
- Always retrieves most current data
- Programmatically reproducible
- Allows filtering and querying
- Demonstrates API data acquisition skills

**Authentication**: None required (public data)

### Step-by-Step Acquisition Instructions

#### Step 1: Open Acquisition Notebook
#### Step 2: Import Required Libraries
#### Step 3: Configure API Request
#### Step 4: Fetch Data with Pagination
The API has a limit on records per request, so we use pagination to fetch all records:
#### Step 5: Convert to DataFrame
#### Step 6: Save to CSV
#### Step 7: Generate SHA-256 Checksum

---

## Dataset 2: Zillow Home Value Index (ZHVI)

### Source Information
- **Provider**: Zillow Research
- **Dataset Name**: ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted
- **URL**: https://www.zillow.com/research/data/
- **Direct Download Link**: https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1762826225
- **License**: Zillow Terms of Use (Attribution required, non-commercial academic use permitted)
- **Update Frequency**: Monthly (typically the 16th)
- **Last Accessed**: 11/10/2025

### Dataset Description
The Zillow Home Value Index (ZHVI) is a smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range.

**Coverage**:
- **Geographic**: All U.S. ZIP codes with sufficient housing data
- **Temporal**: Monthly values from 2000-01-31 to present
- **Housing Types**: Single-family residences, condos, and co-ops

### Acquisition Method: Direct CSV Download

**Why Direct Download?**
- Demonstrates alternative to API access
- Entire dataset available in single file
- No authentication or API key required
- Simpler for static, bulk datasets

**Authentication**: None required

### Step-by-Step Acquisition Instructions

#### Step 1: Open Acquisition Notebook
#### Step 2: Import Required Libraries
#### Step 3: Configure Download
#### Step 4: Download CSV File
#### Step 5: Load and Explore Data
#### Step 6: Generate SHA-256 Checksum

# Storage and Organization Strategy

## Overview
This project uses a **filesystem-based storage approach** for managing raw data. Both datasets are stored as CSV files in an organized directory structure with consistent naming conventions to ensure reproducibility and clarity.

## Storage Model

### Filesystem Storage (CSV Files)
- **Model**: Tabular data stored as CSV files
- **Purpose**: Store original, unprocessed data from different acquisition methods
- **Location**: `data/raw/`
- **Rationale**: Simple, portable, human-readable, and easily version-controlled via checksums

In [13]:
# data cleaning 
RAW_DIR = Path("data/raw")
PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("=== FOOD INSPECTIONS ===")
print(f"Total records: {len(chicago_food_inspection)}")
print(f"Columns: {chicago_food_inspection.columns.tolist()}")
print(f"\nUnique ZIP codes: {chicago_food_inspection['zip'].nunique()}")

print("\n=== ZILLOW ZHVI ===")
print(f"Total records: {len(zhvi)}")
print(f"Cities: {zhvi['City'].value_counts().head()}")
print(f"Chicago records: {len(zhvi[zhvi['City'] == 'Chicago'])}")

=== FOOD INSPECTIONS ===
Total records: 300072
Columns: ['inspection_id', 'dba_name', 'aka_name', 'license_', 'facility_type', 'risk', 'address', 'city', 'state', 'zip', 'inspection_date', 'inspection_type', 'results', 'latitude', 'longitude', 'location.latitude', 'location.longitude', 'location.human_address', 'violations']

Unique ZIP codes: 131

=== ZILLOW ZHVI ===
Total records: 26310
Cities: City
New York       172
Houston        105
Los Angeles     97
San Antonio     57
Chicago         56
Name: count, dtype: int64
Chicago records: 56


# food inspection dataset cleaning

In [16]:
print(f"\n=== MISSING VALUES ===")
missing = chicago_food_inspection.isnull().sum()
print(missing[missing > 0])

print(f"\n=== DUPLICATE ESTABLISHMENTS ===")
print(f"Unique establishments (by license): {chicago_food_inspection['license_'].nunique()}")
print(f"Unique establishments (by DBA name): {chicago_food_inspection['dba_name'].nunique()}")
print(f"Total inspection records: {len(chicago_food_inspection):,}")


=== MISSING VALUES ===
aka_name                    2412
license_                      18
facility_type               5266
risk                          87
city                         166
state                         62
zip                           41
inspection_type                1
latitude                    1031
longitude                   1031
location.latitude           1031
location.longitude          1031
location.human_address    250493
violations                 83755
dtype: int64

=== DUPLICATE ESTABLISHMENTS ===
Unique establishments (by license): 47655
Unique establishments (by DBA name): 34198
Total inspection records: 300,072


In [18]:
print("=== CHECK RISK COLUMN ===\n")
print("Unique values in 'risk' column:")
risk_values = chicago_food_inspection['risk'].value_counts(dropna=False)
print(risk_values)
print(f"\nTotal unique risk values: {chicago_food_inspection['risk'].nunique()}")
print(f"Missing risk values: {chicago_food_inspection['risk'].isnull().sum()}")

=== CHECK RISK COLUMN ===

Unique values in 'risk' column:
risk
Risk 1 (High)      222284
Risk 2 (Medium)     54060
Risk 3 (Low)        23562
NaN                    87
All                    79
Name: count, dtype: int64

Total unique risk values: 4
Missing risk values: 87


In [27]:
food_cleaned = chicago_food_inspection.copy()
food_cleaned = food_cleaned.dropna()

# convert inspection_date to datetime 
food_cleaned['inspection_date'] = pd.to_datetime(food_cleaned['inspection_date'], errors='coerce')
invalid_dates = food_cleaned['inspection_date'].isnull().sum()
if invalid_dates > 0:
    print(f"Removing {invalid_dates} records with invalid dates...")
    food_cleaned = food_cleaned.dropna(subset=['inspection_date'])
print(f"Date range: {food_cleaned['inspection_date'].min()} to {food_cleaned['inspection_date'].max()}")


# keep only the latest inspection for each establishment 

# Check how to identify unique establishments
print("\nIdentifying establishments by:")
print(f"  - DBA name: {food_cleaned['dba_name'].nunique():,} unique")
print(f"  - License number: {food_cleaned['license_'].nunique():,} unique")
print(f"  - Address: {food_cleaned['address'].nunique():,} unique")
# Use a combination of identifiers for best accuracy
# Create a unique establishment ID based on license + address
food_cleaned['establishment_id'] = (
    food_cleaned['license_'].fillna('UNKNOWN') + '_' + 
    food_cleaned['address'].fillna('UNKNOWN')
)

print(f"\nTotal inspection records before: {len(food_cleaned):,}")
print(f"Unique establishments: {food_cleaned['establishment_id'].nunique():,}")

# Sort by inspection date (most recent first) and keep first record per establishment
food_cleaned = food_cleaned.sort_values('inspection_date', ascending=False)
food_cleaned = food_cleaned.groupby('establishment_id', as_index=False).first()
food_latest = food_cleaned.drop(columns=['establishment_id'])
print("\nKept only the latest inspection per establishment")
food_cleaned.shape

Date range: 2010-01-05 00:00:00 to 2025-11-07 00:00:00

Identifying establishments by:
  - DBA name: 14,845 unique
  - License number: 17,235 unique
  - Address: 12,291 unique

Total inspection records before: 42,751
Unique establishments: 17,329

Kept only the latest inspection per establishment


(17329, 20)

In [28]:
print("Cleaning ZIP code column...\n")

# ensure zip is string and 5 digits
food_cleaned['zip'] = food_cleaned['zip'].astype(str).str[:5]
# remove non-numeric characters
food_cleaned['zip'] = food_cleaned['zip'].str.replace(r'\D', '', regex=True)

# remove invalid ZIP codes (empty, less than 5 digits, or starts with 000)
initial_count = len(food_cleaned)
food_cleaned = food_cleaned[
    (food_cleaned['zip'].str.len() == 5) & 
    (~food_cleaned['zip'].str.startswith('000'))
]
removed = initial_count - len(food_cleaned)
print(f"Removed {removed} records with invalid ZIP codes")
print(f"Remaining records: {len(food_cleaned):,}")
print(f"Unique ZIP codes: {food_cleaned['zip'].nunique()}")

print("\nSample ZIP codes:")
print(food_cleaned['zip'].value_counts().head(10))

Cleaning ZIP code column...

Removed 0 records with invalid ZIP codes
Remaining records: 17,329
Unique ZIP codes: 59

Sample ZIP codes:
zip
60614    773
60647    690
60657    625
60618    597
60622    588
60640    568
60625    559
60639    514
60608    513
60611    458
Name: count, dtype: int64


In [32]:
print("Selecting columns for cleaned dataset...\n")
columns_to_keep = [
    'inspection_id',
    'dba_name',
    'aka_name',
    'license_',
    'facility_type',
    'risk',
    'address',
    'city',
    'state',
    'zip',
    'inspection_date',
    'inspection_type',
    'results',
    'violations',
    'latitude',
    'longitude',
    "violations"
]
available_columns = [col for col in columns_to_keep if col in food_cleaned.columns]
food_cleaned = food_cleaned[available_columns].copy()


Selecting columns for cleaned dataset...



In [33]:
print("=== CLEANED DATA SUMMARY ===\n")
print(f"Total records: {len(food_cleaned):,}")
print(f"Unique establishments: {len(food_cleaned):,} (one per establishment)")
print(f"Unique ZIP codes: {food_cleaned['zip'].nunique()}")
print(f"Date range: {food_cleaned['inspection_date'].min()} to {food_cleaned['inspection_date'].max()}")

print("\nRisk distribution:")
print(food_cleaned['risk'].value_counts())

print("\nInspection results distribution:")
print(food_cleaned['results'].value_counts().head(10))

print("\nTop facility types:")
print(food_cleaned['facility_type'].value_counts().head(10))

print("\nMissing values:")
missing = food_cleaned.isnull().sum()
print(missing[missing > 0])

=== CLEANED DATA SUMMARY ===

Total records: 17,329
Unique establishments: 17,329 (one per establishment)
Unique ZIP codes: 59
Date range: 2010-01-05 00:00:00 to 2025-11-07 00:00:00

Risk distribution:
risk
Risk 1 (High)      12633
Risk 2 (Medium)     3249
Risk 3 (Low)        1447
Name: count, dtype: int64

Inspection results distribution:
results
Pass                  8621
Fail                  4596
Pass w/ Conditions    4062
No Entry                48
Not Ready                2
Name: count, dtype: int64

Top facility types:
facility_type
Restaurant                         11649
Grocery Store                       2522
School                               852
Children's Services Facility         546
Bakery                               283
Daycare Above and Under 2 Years      231
Long Term Care                       189
Daycare (2 - 6 Years)                150
Liquor                               104
Catering                              95
Name: count, dtype: int64

Missing values:
S

In [45]:
output_file = PROCESSED_DIR / "food_inspections_cleaned.csv"
food_cleaned.to_csv(output_file, index=False)

with open(output_file, "rb") as f:
    sha = hashlib.sha256(f.read()).hexdigest()

sha_filename = PROCESSED_DIR / "food_inspections_cleaned.sha256"
with open(sha_filename, "w", encoding="utf-8") as f:
    f.write(sha)

# ZHVI dataset cleaning

In [36]:
print("=== DATA PROFILING ===\n")

print(f"Total records: {len(zhvi):,}")
print(f"Total columns: {len(zhvi.columns)}")

print(f"\nFirst 10 columns:")
print(zhvi.columns.tolist()[:10])

print(f"\nData types:")
print(zhvi.dtypes.value_counts())

print(f"\n=== GEOGRAPHIC COVERAGE ===")
print(f"Unique cities: {zhvi['City'].nunique()}")
print(f"Unique states: {zhvi['State'].nunique()}")
print(f"\nTop 10 cities:")
print(zhvi['City'].value_counts().head(10))

=== DATA PROFILING ===

Total records: 26,310
Total columns: 318

First 10 columns:
['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName', '2000-01-31']

Data types:
float64    309
object       6
int64        3
Name: count, dtype: int64

=== GEOGRAPHIC COVERAGE ===
Unique cities: 12718
Unique states: 51

Top 10 cities:
City
New York        172
Houston         105
Los Angeles      97
San Antonio      57
Chicago          56
Philadelphia     50
Dallas           49
Kansas City      48
Springfield      48
Phoenix          46
Name: count, dtype: int64


In [42]:
# ===== ZHVI DATA CLEANING =====
zhvi_cleaned = zhvi.copy()

print("\n=== IDENTIFYING DATE COLUMNS ===")
all_columns = zhvi_cleaned.columns.tolist()
date_columns = [col for col in all_columns if isinstance(col, str) and '-' in col and len(col) == 10]
metadata_columns = [col for col in all_columns if col not in date_columns]
print(f"Total date columns: {len(date_columns)}")
print(f"Date range: {date_columns[0]} to {date_columns[-1]}")
print(f"Metadata columns: {metadata_columns}")

print("\n=== HANDLING MISSING VALUES ===")

# Check missing values in date columns
missing_per_row = zhvi_cleaned[date_columns].isnull().sum(axis=1)
print(f"Missing value statistics per ZIP code:")
print(f"  Mean missing: {missing_per_row.mean():.1f} months")
print(f"  Median missing: {missing_per_row.median():.0f} months")
print(f"  Max missing: {missing_per_row.max():.0f} months")

# remove ZIP codes with >80% missing values
threshold = len(date_columns) * 0.8
initial_count = len(zhvi_cleaned)
zhvi_cleaned = zhvi_cleaned[missing_per_row <= threshold]
removed = initial_count - len(zhvi_cleaned)
print(f"\nRemoved {removed} ZIP codes with >80% missing values")
print(f"Remaining ZIP codes: {len(zhvi_cleaned):,}")


print("\n=== REMOVING ZIP CODES WITH NO RECENT DATA ===")
recent_months = date_columns[-12:]
print(f"Checking recent months: {recent_months[0]} to {recent_months[-1]}")

# Count missing values in recent 12 months
recent_missing = zhvi_cleaned[recent_months].isnull().sum(axis=1)

# Remove ZIP codes with no data in all 12 recent months
initial_count = len(zhvi_cleaned)
zhvi_cleaned = zhvi_cleaned[recent_missing < 12]
removed = initial_count - len(zhvi_cleaned)
print(f"Removed {removed} ZIP codes with NO data in recent 12 months")
print(f"Remaining ZIP codes: {len(zhvi_cleaned):,}")

print("\n=== FINAL DATA SUMMARY ===")

print(f"\nTotal ZIP codes: {len(zhvi_cleaned):,}")
print(f"Total columns: {len(zhvi_cleaned.columns)}")
print(f"Date coverage: {date_columns[0]} to {date_columns[-1]}")


=== IDENTIFYING DATE COLUMNS ===
Total date columns: 309
Date range: 2000-01-31 to 2025-09-30
Metadata columns: ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName']

=== HANDLING MISSING VALUES ===
Missing value statistics per ZIP code:
  Mean missing: 73.5 months
  Median missing: 11 months
  Max missing: 288 months

Removed 1566 ZIP codes with >80% missing values
Remaining ZIP codes: 24,744

=== REMOVING ZIP CODES WITH NO RECENT DATA ===
Checking recent months: 2024-10-31 to 2025-09-30
Removed 0 ZIP codes with NO data in recent 12 months
Remaining ZIP codes: 24,744

=== FINAL DATA SUMMARY ===

Total ZIP codes: 24,744
Total columns: 318
Date coverage: 2000-01-31 to 2025-09-30


In [46]:
output_file = PROCESSED_DIR / "zhvi_cleaned.csv"
zhvi_cleaned.to_csv(output_file, index=False)

with open(output_file, "rb") as f:
    sha = hashlib.sha256(f.read()).hexdigest()

sha_filename = PROCESSED_DIR / "zhvi_cleaned.sha256"
with open(sha_filename, "w", encoding="utf-8") as f:
    f.write(sha)

# Data Cleaning Documentation

## Overview
This section describes the data cleaning process for both the Chicago Food Inspections and Zillow Home Value Index (ZHVI) datasets. Cleaning was performed to ensure data quality, consistency, and readiness for integration and analysis.

---

## Data Cleaning Principles

### Goals
1. **Remove invalid or incomplete records**: Ensure all records have required fields
2. **Standardize formats**: Make data consistent across records
3. **Handle duplicates**: Keep only most relevant records
4. **Prepare for integration**: Ensure join keys are clean and consistent

### General Approach
- **Preserve raw data**: Original files remain unchanged in `data/raw/`
- **Document changes**: All cleaning steps are logged and reproducible
- **Validate results**: Quality checks performed after each cleaning step
- **Generate checksums**: All cleaned files have SHA-256 hashes for verification

---

## Dataset 1: Chicago Food Inspections Cleaning

### Input Data
- **File**: `data/raw/food_inspections.csv`
- **Records**: ~200,000+ inspection records
- **Issues**: Multiple inspections per establishment, null values, inconsistent risk values, invalid ZIP codes

### Cleaning Steps

#### Step 1: Remove All Null Values
**Rationale**: Remove records with any missing values to ensure complete data for analysis
**Impact**: Removes records with missing critical fields

#### Step 2: Convert Inspection Date to Datetime
**Rationale**: 
- Enables date-based sorting and filtering
- Removes records with unparseable dates
**Impact**: Converts dates to datetime objects; removes invalid date formats

#### Step 3: Standardize Risk Column Values
**Rationale**: 
- Ensure consistency in risk categorization
- Remove invalid or ambiguous risk values
- Enable accurate risk-based analysis
**Expected Values**: 
- "Risk 1 (High)"
- "Risk 2 (Medium)"
- "Risk 3 (Low)"
**Impact**: Standardizes risk values; removes records with invalid risk classifications

#### Step 4: Keep Only Latest Inspection Per Establishment
**Rationale**:
- Each establishment should appear once with most current information
- Eliminates redundancy and focuses on current food safety status
- Prevents one establishment from having disproportionate weight in analysis
**Composite Key**: Uses `license_` + `address` because:
- License alone may be missing or non-unique
- Address alone doesn't distinguish multiple businesses at same location
- Combination provides robust identification
**Impact**: Reduces from ~200,000 inspections to ~15,000 unique establishments

#### Step 5: Clean ZIP Code Column
**Rationale**:
- Ensure ZIP codes are valid 5-digit strings
- Remove extended ZIP+4 formats
- Exclude invalid patterns (e.g., 00000)
- Prepare join key for integration
**Impact**: Removes records with invalid ZIP codes; standardizes format

#### Step 6: Select Final Columns
**Rationale**: Keep only relevant columns for analysis

### Output Data
- **File**: `data/processed/food_inspections_cleaned.csv`
- **Records**: ~15,000 establishments (one per unique establishment)
- **Checksum**: `data/processed/food_inspections_cleaned.sha256`

### Data Quality Metrics
- **Completeness**: 100% (all null values removed)
- **Risk values**: 3 unique values (standardized)
- **ZIP codes**: All valid 5-digit format
- **Duplicates**: None (one per establishment)
- **Date range**: From earliest to most recent inspection

---

## Dataset 2: Zillow ZHVI Cleaning

### Input Data
- **File**: `data/raw/zhvi.csv`
- **Records**: ~30,000 US ZIP codes
- **Columns**: ~300+ (metadata + monthly ZHVI values)
- **Issues**: Missing values, stale data, nationwide coverage

### Cleaning Steps

#### Step 1: Identify Date Columns
**Rationale**: Separate time-series data from geographic metadata

#### Step 2: Handle Missing Values

**Rationale**:
- Remove ZIP codes with insufficient historical data
- 80% threshold ensures adequate data for trend analysis
- Preserves ZIPs with minor gaps
**Impact**: Removes ZIP codes lacking sufficient ZHVI history

#### Step 3: Remove ZIP Codes with No Recent Data

**Rationale**:
- Ensure data is current and relevant
- Remove inactive or deprecated ZIP codes
- Focus on areas with active housing markets
**Impact**: Removes stale ZIP codes with no recent housing data

#### Step 4: Keep All Geographic Areas

**Rationale**:
- Maximize potential matches during integration
- Let data determine geographic scope
- Avoid premature filtering

### Output Data
- **File**: `data/processed/zhvi_cleaned.csv`
- **Records**: ~28,000 ZIP codes (nationwide with quality data)
- **Checksum**: `data/processed/zhvi_cleaned.sha256`

### Data Quality Metrics
- **Completeness**: >50% for each ZIP across all months
- **Currency**: All ZIPs have data within last 12 months
- **Coverage**: Nationwide (not pre-filtered)

# data integration

In [52]:
# rename RegionName to zip for consistency
zhvi_cleaned = zhvi_cleaned.rename(columns={'RegionName': 'zip'})

food_cleaned['zip'] = food_cleaned['zip'].astype(str)
zhvi_cleaned['zip'] = zhvi_cleaned['zip'].astype(str)


date_columns = [col for col in zhvi_cleaned.columns if isinstance(col, str) and '-' in col and len(col) == 10]
print(f"\nZHVI date columns: {len(date_columns)} (from {date_columns[0]} to {date_columns[-1]})")
zhvi_cleaned['avg_zhvi_all_time'] = zhvi_cleaned[date_columns].mean(axis=1)
# Average ZHVI for recent years (2020-2024)
recent_date_cols = [col for col in date_columns if col >= '2020-01-01']
zhvi_cleaned['avg_zhvi_recent'] = zhvi_cleaned[recent_date_cols].mean(axis=1)
# Most recent ZHVI (latest month)
zhvi_cleaned['zhvi_latest'] = zhvi_cleaned[date_columns[-1]]
latest_date = date_columns[-1]


housing_columns = [
    'zip',
    'City',
    'State',
    'Metro',
    'CountyName',
    'avg_zhvi_all_time',
    'avg_zhvi_recent',
    'zhvi_latest'
]
available_housing_cols = [col for col in housing_columns if col in zhvi_cleaned.columns]
zhvi_for_merge = zhvi_cleaned[available_housing_cols].copy()


ZHVI date columns: 309 (from 2000-01-31 to 2025-09-30)


In [53]:
print("\n=== PERFORMING INTEGRATION ===")

# Check for overlap before merge
food_zips = set(food_cleaned['zip'].unique())
housing_zips = set(zhvi_for_merge['zip'].unique())
common_zips = food_zips & housing_zips

print(f"\nZIP code overlap analysis:")
print(f"  Food inspection ZIPs: {len(food_zips)}")
print(f"  Housing ZIPs (all): {len(housing_zips)}")
print(f"  ZIPs in BOTH datasets: {len(common_zips)}")
print(f"  Food ZIPs not in housing: {len(food_zips - housing_zips)}")
print(f"  Housing ZIPs not in food: {len(housing_zips - food_zips)}")

# Perform INNER JOIN - keep only ZIP codes present in BOTH datasets
integrated_data = pd.merge(
    food_cleaned,
    zhvi_for_merge,
    on='zip',
    how='inner',
    suffixes=('_food', '_housing')
)

print(f"\nIntegration complete!")
print(f"Integrated records: {len(integrated_data):,}")
print(f"Integrated ZIP codes: {integrated_data['zip'].nunique()}")


=== PERFORMING INTEGRATION ===

ZIP code overlap analysis:
  Food inspection ZIPs: 59
  Housing ZIPs (all): 24744
  ZIPs in BOTH datasets: 58
  Food ZIPs not in housing: 1
  Housing ZIPs not in food: 24686

Integration complete!
Integrated records: 17,141
Integrated ZIP codes: 58


In [57]:
integrated_data.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,longitude,violations,violations,City,State,Metro,CountyName,avg_zhvi_all_time,avg_zhvi_recent,zhvi_latest
0,2576806,MINI SUPER DANY,MINI SUPER DANY,0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617,...,-87.53513317727304,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...","1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",Chicago,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,107135.791904,159044.895191,170357.689869
1,2362220,TAMALES Y RESTAURANT COSTA GRANDE,TAMALES Y RESTAURANT COSTA GRANDE,0,Restaurant,Risk 1 (High),10148 S EWING AVE,CHICAGO,IL,60617,...,-87.53542925902096,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,Chicago,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,107135.791904,159044.895191,170357.689869
2,2282248,LITTLE BLACK PEARL,LITTLE BLACK PEARL,0,School,Risk 2 (Medium),1060 E 47TH ST,CHICAGO,IL,60653,...,-87.59938918300288,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...","55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",Chicago,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,274492.806385,328633.728627,320980.029247
3,2546857,KRISPY KREME DOUGHNUTS,KRISPY KREME DOUGHNUTS,0,Restaurant,Risk 2 (Medium),108 N STATE ST,CHICAGO,IL,60602,...,-87.62802165207536,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,Chicago,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,423684.595837,346178.467704,257735.194584
4,2545091,OMBUDSMAN ROSELAND,OMBUDSMAN ROSELAND,0,School,Risk 3 (Low),10928 S HALSTED ST,CHICAGO,IL,60628,...,-87.64261132457379,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,Chicago,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,91384.197397,141288.778561,145351.509384


In [60]:
print("\n=== POST-INTEGRATION ANALYSIS ===")

# Summary statistics
print(f"\nIntegrated Dataset:")
print(f"  Total establishments: {len(integrated_data):,}")
print(f"  Unique ZIP codes: {integrated_data['zip'].nunique()}")
print(f"  Columns: {len(integrated_data.columns)}")

# Establishments per ZIP
establishments_per_zip = integrated_data.groupby('zip').size()
print(f"\nEstablishments per ZIP code:")
print(f"  Mean: {establishments_per_zip.mean():.1f}")
print(f"  Median: {establishments_per_zip.median():.0f}")
print(f"  Min: {establishments_per_zip.min()}")
print(f"  Max: {establishments_per_zip.max()}")


=== POST-INTEGRATION ANALYSIS ===

Integrated Dataset:
  Total establishments: 17,141
  Unique ZIP codes: 58
  Columns: 26

Establishments per ZIP code:
  Mean: 295.5
  Median: 282
  Min: 14
  Max: 773


In [61]:
# Risk distribution
print(f"\nRisk distribution in integrated data:")
print(integrated_data['risk'].value_counts())

# Housing values
print(f"\nHousing value statistics (latest ZHVI):")
print(f"  Mean: ${integrated_data['zhvi_latest'].mean():,.0f}")
print(f"  Median: ${integrated_data['zhvi_latest'].median():,.0f}")
print(f"  Min: ${integrated_data['zhvi_latest'].min():,.0f}")
print(f"  Max: ${integrated_data['zhvi_latest'].max():,.0f}")

# Top facility types
print(f"\nTop 5 facility types in integrated data:")
print(integrated_data['facility_type'].value_counts().head())


Risk distribution in integrated data:
risk
Risk 1 (High)      12540
Risk 2 (Medium)     3181
Risk 3 (Low)        1420
Name: count, dtype: int64

Housing value statistics (latest ZHVI):
  Mean: $348,064
  Median: $332,782
  Min: $122,679
  Max: $612,923

Top 5 facility types in integrated data:
facility_type
Restaurant                      11491
Grocery Store                    2496
School                            852
Children's Services Facility      546
Bakery                            283
Name: count, dtype: int64


In [62]:
output_file = PROCESSED_DIR / "integrated_food_housing.csv"
integrated_data.to_csv(output_file, index=False)

with open(output_file, "rb") as f:
    sha = hashlib.sha256(f.read()).hexdigest()

sha_filename = PROCESSED_DIR / "integrated_food_housing.sha256"
with open(sha_filename, "w", encoding="utf-8") as f:
    f.write(sha)

# Data Integration Documentation

## Overview
This section describes the integration of cleaned Chicago Food Inspections and Zillow Home Value Index (ZHVI) datasets. The integration creates an establishment-level dataset linking food safety metrics with neighborhood housing values.

---

## Integration Strategy

### Objective
Create a unified dataset where each food establishment is associated with housing market characteristics of its ZIP code area.

### Approach
**Join Type**: INNER JOIN on ZIP code  
**Granularity**: Establishment-level (one row per food establishment)  
**Geography**: Determined by data overlap 

### Key Principle
Let the data determine geographic scope by joining on exact ZIP code matches without pre-filtering by city.

---

## Conceptual Model

### Entity-Relationship Diagram (Insert here)

### Data flow (insert here)

---

## Integration Schema

### Source 1: Food Inspections (Cleaned)
**File**: `data/processed/food_inspections_cleaned.csv`  
**Key**: `zip` (string, 5 digits)  
**Granularity**: Establishment-level

### Source 2: ZHVI (Cleaned)
**File**: `data/processed/zhvi_cleaned.csv`  
**Key**: `zip` (renamed from RegionName)  
**Granularity**: ZIP code-level


---

### Housing Metrics Calculation
**Rationale**:
- **all_time**: Historical perspective on housing values
- **recent**: Current market conditions (last 5 years)
- **latest**: Most up-to-date snapshot